In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import scipy.io as io

pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

In [7]:
PATH = 'ml/results_rf_300922_rich_region_ncx_imp.csv'

In [8]:
results = pd.read_csv(PATH, index_col=0)
complete = results[results.restriction == 'complete']
complete = complete.dropna(how='all', axis=1)
grouped_complete = complete.groupby(by=['restriction', 'modality', 'chunk_size'])
grouped_complete[['auc', 'dev_auc']].quantile(q=[0.25, 0.5, 0.75])

auc   dev_auc
restriction modality      chunk_size                         
complete    morphological 50.0       0.25  0.979107  0.931548
                                     0.50  0.989473  1.000000
                                     0.75  0.993479  1.000000
            spatial       25.0       0.25  0.859633  0.833333
                                     0.50  0.892711  0.928571
                                     0.75  0.917919  1.000000
            temporal      1600.0     0.25  0.835152  0.785714
                                     0.50  0.856690  0.857143
                                     0.75  0.871988  0.928571

In [6]:
wf_ca1 = complete[complete.modality == 'morphological'].dev_auc.to_numpy()
wf_ncx = complete[complete.modality == 'morphological'].auc.to_numpy()

temp_ca1 = complete[complete.modality == 'temporal'].dev_auc.to_numpy()
temp_ncx = complete[complete.modality == 'temporal'].auc.to_numpy()

spat_ca1 = complete[complete.modality == 'spatial'].dev_auc.to_numpy()
spat_ncx = complete[complete.modality == 'spatial'].auc.to_numpy()

statistic, p_val = stats.mannwhitneyu(wf_ca1, wf_ncx)
print(f"Mann-Whitney statistical test results for WF are p-value={p_val} (statistic={statistic})")
print()

statistic, p_val = stats.mannwhitneyu(temp_ca1, temp_ncx)
print(f"Mann-Whitney statistical test results for temporal are p-value={p_val} (statistic={statistic})")
print()

statistic, p_val = stats.mannwhitneyu(spat_ca1, spat_ncx)
print(f"Mann-Whitney statistical test results for spatial are p-value={p_val} (statistic={statistic})")
print()

Mann-Whitney statistical test results for WF are p-value=0.0031361807951584427 (statistic=855.0)

Mann-Whitney statistical test results for temporal are p-value=3.468263926972842e-18 (statistic=0.0)

Mann-Whitney statistical test results for spatial are p-value=4.21486133229539e-18 (statistic=3.0)



In [5]:
cs_50 = complete[complete.chunk_size == 50]
cs_wf = cs_50[cs_50.modality == 'morphological']
wf_aucs = cs_wf.auc.to_numpy()
wf_dev_aucs = cs_wf.dev_auc.to_numpy()

cs_1600 = complete[complete.chunk_size == 1600]
cs_temp = cs_1600[cs_1600.modality == 'temporal']
temp_aucs = cs_temp.auc.to_numpy()
temp_dev_aucs = cs_temp.dev_auc.to_numpy()

cs_25 = complete[complete.chunk_size == 25]
cs_spat = cs_25[cs_25.modality == 'spatial']
spat_aucs = cs_spat.auc.to_numpy()
spat_dev_aucs = cs_spat.dev_auc.to_numpy()

aucs = np.concatenate((wf_aucs, temp_aucs, spat_aucs))
dev_aucs = np.concatenate((wf_dev_aucs, temp_dev_aucs, spat_dev_aucs))
N = len(aucs)

all_aucs = np.concatenate((aucs, dev_aucs))
g1 = np.concatenate((np.ones(N), 2 * np.ones(N)))
g2 = np.repeat([[1, 2, 3, 1, 2, 3]], N // 3, axis=1).flatten()

In [6]:
m_dict = {'data': all_aucs, 'g1': g1, 'g2': g2}
io.savemat('region_mat.mat', m_dict)

In [7]:
wf_diff = 100 * (wf_dev_aucs - wf_aucs) / wf_dev_aucs
temp_diff = 100 * (temp_dev_aucs - temp_aucs) / temp_dev_aucs
spat_diff = 100 * (spat_dev_aucs - spat_aucs) / spat_dev_aucs

wf_median, wf_prec25, wf_prec75 = np.percentile(wf_diff, [50, 25, 75])
print(f"Median of WF column is {wf_median: .3f} [{wf_prec25: .3f}, {wf_prec75: .3f}]")
temp_median, temp_prec25, temp_prec75 = np.percentile(temp_diff, [50, 25, 75])
print(f"Median of temporal column is {temp_median: .3f} [{temp_prec25: .3f}, {temp_prec75: .3f}]")
spat_median, spat_prec25, spat_prec75 = np.percentile(spat_diff, [50, 25, 75])
print(f"Median of spatial column is {spat_median: .3f} [{spat_prec25: .3f}, {spat_prec75: .3f}]\n")

statistic, p_val = stats.wilcoxon(wf_dev_aucs, wf_aucs, alternative='greater')
print(f"Wilcoxon statistical test results for wf are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(spat_dev_aucs, spat_aucs, alternative='greater')
print(f"Wilcoxon statistical test results for spatial are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(temp_dev_aucs, temp_aucs, alternative='greater')
print(f"Wilcoxon statistical test results for temporal p-value={p_val} (statistic={statistic})\n")

statistic, p_val = stats.wilcoxon(wf_diff, temp_diff)
print(f"Wilcoxon statistical test results for wf and temp are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(wf_diff, spat_diff)
print(f"Wilcoxon statistical test results for wf and spatial are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(spat_diff, temp_diff)
print(f"Wilcoxon statistical test results for temporal and spatial p-value={p_val} (statistic={statistic})")

Median of WF column is  1.771 [ 0.125,  8.337]
Median of temporal column is  40.808 [ 28.569,  50.778]
Median of spatial column is  27.525 [ 20.494,  33.974]

Wilcoxon statistical test results for wf are p-value=1.6212069488518604e-05 (statistic=1068.0)
Wilcoxon statistical test results for spatial are p-value=3.778464727931783e-10 (statistic=1275.0)
Wilcoxon statistical test results for temporal p-value=4.0155454535234565e-10 (statistic=1274.0)

Wilcoxon statistical test results for wf and temp are p-value=1.0872449061578735e-09 (statistic=6.0)
Wilcoxon statistical test results for wf and spatial are p-value=9.06805772149548e-10 (statistic=3.0)
Wilcoxon statistical test results for temporal and spatial p-value=0.0003887877875099616 (statistic=270.0)


In [8]:
statistic, p_val = stats.wilcoxon(wf_aucs - 0.5)
print(f"Mann-Whitney statistical test results for wf ncx auc are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(spat_aucs - 0.5)
print(f"Mann-Whitney statistical test results for spatial ncx auc are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(temp_aucs - 0.5)
print(f"Mann-Whitney statistical test results for temporal ncx auc are p-value={p_val} (statistic={statistic})")

statistic, p_val = stats.wilcoxon(wf_dev_aucs - 0.5)
print(f"Mann-Whitney statistical test results for wf ca1 auc are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(spat_dev_aucs - 0.5)
print(f"Mann-Whitney statistical test results for spatial ca1 auc are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(temp_dev_aucs - 0.5)
print(f"Mann-Whitney statistical test results for temporal ca1 auc are p-value={p_val} (statistic={statistic})")

Mann-Whitney statistical test results for wf ncx auc are p-value=7.544976683806673e-10 (statistic=0.0)
Mann-Whitney statistical test results for spatial ncx auc are p-value=1.431736905499615e-07 (statistic=92.5)
Mann-Whitney statistical test results for temporal ncx auc are p-value=0.2839328003398659 (statistic=526.5)
Mann-Whitney statistical test results for wf ca1 auc are p-value=7.544976683806673e-10 (statistic=0.0)
Mann-Whitney statistical test results for spatial ca1 auc are p-value=7.551804758280675e-10 (statistic=0.0)
Mann-Whitney statistical test results for temporal ca1 auc are p-value=7.551804758280675e-10 (statistic=0.0)


In [9]:
statistic, p_val = stats.wilcoxon(temp_aucs, spat_aucs)
print(f"Wilcoxon statistical test results for spatial and temporal are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(temp_aucs, wf_aucs)
print(f"Wilcoxon statistical test results for wf and temporal are p-value={p_val} (statistic={statistic})")

Wilcoxon statistical test results for spatial and temporal are p-value=0.0007413258442797535 (statistic=288.0)
Wilcoxon statistical test results for wf and temporal are p-value=8.031090907046913e-10 (statistic=1.0)
